In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image  # Para crear el GIF
import random
import os
import math
from scipy.spatial import distance_matrix

# Configuración de parámetros
POPULATION_SIZE = 200
GENERATIONS = 50000
MUTATION_RATE = 0.4
ELITE_SIZE = 5
SAVE_INTERVAL = 1000  # Intervalo de generación para guardar imágenes

# Cargar datos desde el archivo .txt
def load_tsp_data(file_path):
    coords = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        start_index = lines.index('NODE_COORD_SECTION\n') + 1
        for line in lines[start_index:]:
            if line.strip() == 'EOF':
                break
            _, x, y = line.split()
            coords.append((float(x), float(y)))
    return np.array(coords)

# Generar una ruta aleatoria
def create_route(cities):
    return random.sample(list(cities), len(cities))

# Calcular la distancia de la ruta
def route_distance(route, dist_matrix):
    return sum(dist_matrix[route[i], route[i + 1]] for i in range(len(route) - 1)) + dist_matrix[route[-1], route[0]]

# Crear la población inicial
def create_population(pop_size, cities):
    return [create_route(cities) for _ in range(pop_size)]

# Selección de los mejores individuos
def rank_routes(population, dist_matrix):
    fitness_results = {i: route_distance(ind, dist_matrix) for i, ind in enumerate(population)}
    return sorted(fitness_results.items(), key=lambda x: x[1])

# Selección por torneo
def selection(ranked_population):
    selected = []
    for i in range(ELITE_SIZE):
        selected.append(ranked_population[i][0])
    for i in range(len(ranked_population) - ELITE_SIZE):
        selected.append(random.choice(ranked_population[:POPULATION_SIZE // 2])[0])
    return selected

# Cruzar dos padres
def breed(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = parent1[start:end]
    child += [item for item in parent2 if item not in child]
    return child

# Aplicar mutación 2-Opt
def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        i, j = sorted(random.sample(range(len(individual)), 2))
        individual[i:j] = individual[i:j][::-1]  # Inversión de una sección
    return individual

# Crear la nueva generación
def evolve_population(population, dist_matrix):
    ranked_population = rank_routes(population, dist_matrix)
    selected_indices = selection(ranked_population)
    selected_population = [population[i] for i in selected_indices]
    children = [breed(selected_population[i], selected_population[-i-1]) for i in range(len(selected_population) - ELITE_SIZE)]
    mutated_population = [mutate(ind, MUTATION_RATE) for ind in children]
    return selected_population[:ELITE_SIZE] + mutated_population

# Guardar una imagen de la ruta
def save_route_image(cities, route, generation, distance):
    x = [cities[city][0] for city in route] + [cities[route[0]][0]]
    y = [cities[city][1] for city in route] + [cities[route[0]][1]]
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, 'o-', markersize=5, lw=2)
    plt.title(f"Generación {generation} - Distancia óptima: {distance:.2f}")
    plt.xlabel("Coordenada X")
    plt.ylabel("Coordenada Y")
    plt.savefig(f"progress_images/progress_generation_{generation}.png")
    plt.close()

# Crear un GIF a partir de las imágenes guardadas
def create_gif(output_path='tsp_progress.gif'):
    images = []
    for filename in sorted(os.listdir('progress_images'), key=lambda x: int(x.split('_')[2].split('.')[0]) if 'generation' in x else float('inf')):
        if filename.endswith('.png') and 'generation' in filename:
            file_path = os.path.join('progress_images', filename)
            images.append(Image.open(file_path))
    if images:
        images[0].save(output_path, save_all=True, append_images=images[1:], duration=300, loop=0)
        print(f"GIF creado exitosamente: {output_path}")
    else:
        print("No se encontraron imágenes para crear el GIF.")

# Guardar progreso en archivo .txt
def save_progress_txt(generation, distance, route):
    with open("tsp_progress.txt", "a") as file:
        file.write(f"Generación {generation} - Distancia óptima: {distance:.2f}\n")
        file.write(f"Recorrido: {route}\n\n")

# Ejecutar el algoritmo
def genetic_algorithm_tsp(file_path):
    cities = load_tsp_data(file_path)
    dist_matrix = distance_matrix(cities, cities)
    population = create_population(POPULATION_SIZE, range(len(cities)))

    # Crear carpeta para guardar imágenes si no existe
    if not os.path.exists('progress_images'):
        os.makedirs('progress_images')

    # Limpiar el archivo de registro al iniciar
    open("tsp_progress.txt", "w").close()

    for generation in range(GENERATIONS):
        population = evolve_population(population, dist_matrix)
        best_route_idx, best_distance = rank_routes(population, dist_matrix)[0]
        best_route = population[best_route_idx]
        
        # Guardar cada SAVE_INTERVAL generaciones para la animación y el log
        if generation % SAVE_INTERVAL == 0 or generation == GENERATIONS - 1:
            save_route_image(cities, best_route, generation, best_distance)
            save_progress_txt(generation, best_distance, best_route)
            print(f"Generación {generation} - Distancia óptima: {best_distance}")

    # Crear el GIF al finalizar
    create_gif()
    print("Tour óptimo encontrado:", best_route)
    print("Distancia total:", best_distance)

# Ruta al archivo .txt
file_path = 'ch150.tsp'
genetic_algorithm_tsp(file_path)

Generación 0 - Distancia óptima: 48606.195240506655
Generación 1000 - Distancia óptima: 18750.503500271316
Generación 2000 - Distancia óptima: 17326.514086406183
Generación 3000 - Distancia óptima: 15541.492484990893
Generación 4000 - Distancia óptima: 14989.700508634362
Generación 5000 - Distancia óptima: 14574.118463493962
Generación 6000 - Distancia óptima: 13924.068895684406
Generación 7000 - Distancia óptima: 13821.915262195647
Generación 8000 - Distancia óptima: 13796.282954938773
Generación 9000 - Distancia óptima: 13488.783340092157
Generación 10000 - Distancia óptima: 13408.827395300554
Generación 11000 - Distancia óptima: 13309.033762355095
Generación 12000 - Distancia óptima: 13291.353746754932
Generación 13000 - Distancia óptima: 13289.826568530974
Generación 14000 - Distancia óptima: 13289.826568530974
Generación 15000 - Distancia óptima: 13277.804893881465
Generación 16000 - Distancia óptima: 13277.804893881465
Generación 17000 - Distancia óptima: 13101.334755835593
Gener